## 使用线性判别分析（LDA）在鸢尾花数据集上进行数据规约

In [2]:
# 导入鸢尾花数据集
from sklearn.datasets import load_iris

In [13]:
# 导入一些必要的库
import numpy as np

In [7]:
# 加载数据集
iris = load_iris() 
# 分别存储数据集的行（样本）和列（特征）
iris_X, iris_Y = iris.data, iris.target

In [9]:
# 查看数据的size
iris_X.shape, iris_Y.shape

((150, 4), (150,))

In [10]:
# 查看数据的类别
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [11]:
# 查看数据的特征 
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [16]:
# 将类别名与标签一一映射
label_dict = {i:k for i,k in enumerate(iris.target_names)}

In [17]:
# 查看映射
label_dict

{0: 'setosa', 1: 'versicolor', 2: 'virginica'}

### LDA大致分为以下五个步骤
1. 计算每个类别的均值向量；
2. 计算类内和类间的散布矩阵；
3. 计算 $S_W^{-1}$·$S_B$的特征值和特征向量；
4. 降序排列特征值，保留前k个特征向量；
5. 使用前几个特征向量将数据投影到新空间。

#### 1. 计算每个类别的均值向量 

In [21]:
mean_vectors = []
for cls in [0, 1, 2]:
    class_mean_vector = np.mean(iris_X[iris_Y==cls],axis=0)
    mean_vectors.append(class_mean_vector)
    print(label_dict[cls], class_mean_vector)


setosa [5.006 3.428 1.462 0.246]
versicolor [5.936 2.77  4.26  1.326]
virginica [6.588 2.974 5.552 2.026]


#### 2. 计算类内和类间的散布矩阵 

类内散布矩阵，定义如下：
                    $S_W$ = $\sum_{i=1}^c$ $S_i$

$S_i$的定义是：
                    $S_i$ = $\sum_{x\in D_i}^n$ $(x - m_i)(x-m_i)^T$

其中，$m_i$代表第 $i$ 个类别的均值向量。

类间散布矩阵的定义是：
                    $S_B$ = $\sum_{i=1}^c$ $N_i(m_i-m)(m_i-m)^T$


$m$是数据集的总体均值，$m_i$是每个类别的样本均值，$N_i$是每个类别的样本大小（观察值数量）

In [22]:
# 类内散布矩阵
S_W = np.zeros((4,4))
for cls, mv in zip(range(3), mean_vectors):
    S_cls = np.zeros((4,4))
    for row in iris_X[iris_Y == cls]:
        row, mv = row.reshape(4,1), mv.reshape(4,1)
        S_cls += (row-mv).dot((row-mv).T)
    S_W += S_cls
    
print(S_W)

[[38.9562 13.63   24.6246  5.645 ]
 [13.63   16.962   8.1208  4.8084]
 [24.6246  8.1208 27.2226  6.2718]
 [ 5.645   4.8084  6.2718  6.1566]]


In [40]:
# 类间散布矩阵
all_mean = np.mean(iris_X, axis=0).reshape(4,1)
S_B = np.zeros((4,4))
for cls, mv in zip(range(3), mean_vectors):
    mv = mv.reshape(4,1)
    S_B += len(iris_X[iris_Y == cls])*(mv-all_mean).dot((mv-all_mean).T)

print(S_B)

[[ 63.21213333 -19.95266667 165.2484      71.27933333]
 [-19.95266667  11.34493333 -57.2396     -22.93266667]
 [165.2484     -57.2396     437.1028     186.774     ]
 [ 71.27933333 -22.93266667 186.774       80.41333333]]


#### 3. 计算 $S_W^{-1}$·$S_B$的特征值和特征向量；

In [55]:
eig_vals, eig_vecs = np.linalg.eig(np.dot(np.linalg.inv(S_W), S_B))

for i in range(len(eig_vals)):     
    eig_vec_cls = eig_vecs[:,i]     
    print('特征向量 {}: {}'.format(i+1, eig_vec_cls))     
    print('特征值 {}: {}'.format(i+1, eig_vals[i])) 

特征向量 1: [ 0.20874182  0.38620369 -0.55401172 -0.7073504 ]
特征值 1: 32.19192919827804
特征向量 2: [-0.00653196 -0.58661055  0.25256154 -0.76945309]
特征值 2: 0.2853910426230688
特征向量 3: [ 0.56419916  0.07377204  0.14243606 -0.80990676]
特征值 3: 4.959432195627595e-15
特征向量 4: [-0.50482739  0.4425648   0.48738477 -0.55833842]
特征值 4: -5.788395204902564e-15


#### 4. 降序排列特征值，保留前k个特征向量